In [10]:
from __future__ import print_function
import warnings
warnings.filterwarnings('ignore')
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from keras.utils.np_utils import to_categorical
import cv2
import os
from tqdm.notebook import tqdm
from tensorflow.keras.models import Sequential, Model
from tensorflow.keras.layers import Dense, Dropout, Activation, Flatten
from tensorflow.keras.layers import Conv2D, MaxPooling2D, ZeroPadding2D
from tensorflow.keras.layers import BatchNormalization
from keras.regularizers import l2
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras import optimizers
import tensorflow.keras as keras
import tensorflow.keras.backend as K
from tensorflow.keras.models import load_model

In [11]:
data = 'ecg_data'
train_data_path = 'ecg_data\mitbih_train.csv'
test_data_path = 'ecg_data\mitbih_test.csv'
Classes: {'N': 0, 'S': 1, 'V': 2, 'F': 3, 'Q': 4}

In [12]:
train_df = pd.read_csv(train_data_path,header=None)
test_df = pd.read_csv(test_data_path, header=None)

In [4]:
train_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 87554 entries, 0 to 87553
Columns: 188 entries, 0 to 187
dtypes: float64(188)
memory usage: 125.6 MB


In [5]:
train_df[187]=train_df[187].astype(int)

In [6]:
equilibre=train_df[187].value_counts()
print(equilibre)

0    72471
4     6431
2     5788
1     2223
3      641
Name: 187, dtype: int64


In [23]:
#saving data
# for index, row in tqdm(train_df.iterrows()):
#     if index%50==0:
#         img = row[:187]
#         label = row[187]
#         plt.clf()
#         plt.plot(img)
#         plt.savefig(os.path.join(str(int(label)),str(index)+'.jpg'))
#     plt.show()
#         print("label", label)
#     if index == 100:
#         break;
    

In [24]:
# for index, row in tqdm(train_df.iterrows()):
#     if index%50==0:
#         img = row[:187]
#         label = row[187]
#         plt.clf()
#         plt.plot(img)
#         path_to_save = os.path.join('ecg_data',os.path.join(str(int(label)),str(index)+'.jpg'))
#         plt.savefig(path_to_save)
#         img = cv2.imread(path_to_save)
#         img = img[36:-36,55:-53,:]
#         cv2.imwrite(path_to_save, img)

In [2]:
 #y_test.shape

In [4]:
# y_test[15001]

In [16]:

# settings 
img_height  = 128
img_width  = 128 
img_channel = 1
batch_size  = 2
nb_classes = 5
epochs = 30

trainset = 'train'
# testset = './dataset/test'
# validset = './dataset/valid'


In [17]:
train_datagen = ImageDataGenerator(rescale=1./255,validation_split=0.15)

In [18]:
# splitting
train_generator = train_datagen.flow_from_directory(
    trainset,
    target_size=(img_height, img_width),
    batch_size=batch_size,
    subset='training',class_mode='categorical', color_mode ='grayscale') # set as training data

validation_generator = train_datagen.flow_from_directory(
    trainset,
    target_size=(img_height, img_width),
    batch_size=batch_size,
    subset='validation',class_mode='categorical',color_mode ='grayscale') # set as validation data

Found 747 images belonging to 5 classes.
Found 129 images belonging to 5 classes.


In [19]:


def alexnet_model(img_shape=(224, 224, 3), n_classes=5, l2_reg=0.,weights=None):

    # Initialize model
    alexnet = Sequential()

    # Layer 1
    alexnet.add(Conv2D(16, (11, 11), input_shape=img_shape, padding='same', kernel_regularizer=l2(l2_reg)))
    alexnet.add(BatchNormalization())
    alexnet.add(Activation('relu'))
    alexnet.add(MaxPooling2D(pool_size=(2, 2)))

    # Layer 2
    alexnet.add(Conv2D(23, (5, 5), padding='same'))
    alexnet.add(BatchNormalization())
    alexnet.add(Activation('relu'))
    alexnet.add(MaxPooling2D(pool_size=(2, 2)))

    # Layer 3
    alexnet.add(ZeroPadding2D((1, 1)))
    alexnet.add(Conv2D(64, (3, 3), padding='same'))
    alexnet.add(BatchNormalization())
    alexnet.add(Activation('relu'))
    alexnet.add(MaxPooling2D(pool_size=(2, 2)))

    # Layer 4
    alexnet.add(ZeroPadding2D((1, 1)))
    alexnet.add(Conv2D(128, (3, 3), padding='same'))
    alexnet.add(BatchNormalization())
    alexnet.add(Activation('relu'))

    # Layer 5
    alexnet.add(ZeroPadding2D((1, 1)))
    alexnet.add(Conv2D(256, (3, 3), padding='same'))
    alexnet.add(BatchNormalization())
    alexnet.add(Activation('relu'))
    alexnet.add(MaxPooling2D(pool_size=(2, 2)))

    # Layer 6
    alexnet.add(Flatten())
    alexnet.add(Dense(512))
    alexnet.add(BatchNormalization())
    alexnet.add(Activation('relu'))
    alexnet.add(Dropout(0.5))

    # Layer 7
    alexnet.add(Dense(1024))
    alexnet.add(BatchNormalization())
    alexnet.add(Activation('relu'))
    alexnet.add(Dropout(0.5))

    # Layer 8
    alexnet.add(Dense(n_classes))
    alexnet.add(BatchNormalization())
    alexnet.add(Activation('softmax'))

    if weights is not None:
        alexnet.load_weights(weights)

    return alexnet


In [20]:
alex = alexnet_model(img_shape=(128, 128, 1), n_classes=5, l2_reg=0.,weights=None)


Instructions for updating:
Call initializer instance with the dtype argument instead of passing it to the constructor


In [21]:
alex.compile(loss='categorical_crossentropy', optimizer='adam',metrics=['accuracy'])

In [23]:
history = alex.fit_generator(
    train_generator,
    steps_per_epoch=train_generator.samples // batch_size,
    validation_data=validation_generator,
    validation_steps=validation_generator.samples // batch_size,
    epochs=30,verbose=1)

Epoch 1/30
373/373 [==============================] - 148s 396ms/step - loss: 0.8068 - acc: 0.8255 - val_loss: 0.7205 - val_acc: 0.8359
Epoch 2/30
373/373 [==============================] - 152s 409ms/step - loss: 0.7518 - acc: 0.8268 - val_loss: 1.4594 - val_acc: 0.8359
Epoch 3/30
373/373 [==============================] - 147s 395ms/step - loss: 0.6985 - acc: 0.8282 - val_loss: 0.7508 - val_acc: 0.8125
Epoch 4/30
373/373 [==============================] - 147s 395ms/step - loss: 0.6919 - acc: 0.8255 - val_loss: 1.0166 - val_acc: 0.8125
Epoch 5/30
373/373 [==============================] - 146s 391ms/step - loss: 0.6873 - acc: 0.8242 - val_loss: 0.6021 - val_acc: 0.8359
Epoch 6/30
373/373 [==============================] - 146s 393ms/step - loss: 0.6678 - acc: 0.8255 - val_loss: 1.3950 - val_acc: 0.6406
Epoch 7/30
373/373 [==============================] - 138s 371ms/step - loss: 0.6469 - acc: 0.8282 - val_loss: 0.6325 - val_acc: 0.8516
Epoch 8/30
373/373 [============================